In [ ]:
%load_ext autotime

In [ ]:
import torch
from transformers import BertTokenizer
import json
from collections import Counter
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import metrics
from xgboost import XGBClassifier

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import numpy as np
import copy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import stats
import seaborn as sns
from scipy import stats
from sklearn import metrics
from gensim.models import Word2Vec

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
df_annotations=  pd.read_csv('C:/Users/u0146965/OneDrive - KU Leuven/Sentiment analysis/full_cleaned_dataset.csv')
df_annotations# to display the first 5 lines of loaded data

# XGBoost met TFIDF

In [ ]:
#doe TIDF op de data
def TFIDF(data, max_feat, n_gram):
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(data)
    X= vect.fit_transform(data)
    X=pd.DataFrame(X.toarray(), columns= vect.get_feature_names())
    return X

In [ ]:
#zet de y-waarden goed
def y_set(dataset_column):
    y= dataset_column
    y = y.replace('NEUTRAL','1')
    y = y.replace('POSITIVE','2')
    y = y.replace('NEGATIVE','0')
    for i in y:
        i=int(i)
    new=[]
    for i in y:
        new.append(int(i))
    
    return new

In [ ]:
#definieer de confusion matrix
def cf_mat(y_test, y_pred):
    mat=confusion_matrix(y_test, y_pred)/len(y_test)
    return sns.heatmap(mat, annot=True)

In [ ]:
def XgBoost(X, y_inputkolom, max_feat, n_gram, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    #do TFIDF
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(X_train)
    X_train= vect.transform(X_train)
    X_test= vect.transform(X_test)
    X_train=pd.DataFrame(X_train.toarray(), columns= vect.get_feature_names())
    X_test=pd.DataFrame(X_test.toarray(), columns= vect.get_feature_names())
    
    # Build XGBoost
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = xg_boost.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', accuracy_score(y_test, y_predict))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    #return y_predict, y_test, log_reg

In [ ]:
def XgBoost_val_tuning(X,y_inputkolom, max_feat, n_gram, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    vect=TfidfVectorizer(max_features=max_feat, stop_words= stopwords.words('dutch'), token_pattern=r'\b[^\d\W][^\d\W]+\b', ngram_range=n_gram).fit(X_train)
    X_train= vect.transform(X_train)
    X_val= vect.transform(X_val)
    X_train=pd.DataFrame(X_train.toarray(), columns= vect.get_feature_names())
    X_val=pd.DataFrame(X_val.toarray(), columns= vect.get_feature_names())

    # Build a logistic regression
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost= model.fit(X_train, y_train)

    # Predict the labels 
    y_predict = xg_boost.predict(X_val)
    acc_score= accuracy_score(y_val, y_predict)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

# TFIDF

## Textified

In [ ]:
plotje=[]
max_feat_list=[1000, 2000, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'], i, (1,1))
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],4000, i)

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
for i in max_depth:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],4000, (1,1) ,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
plt.xlabel('max_features')
plt.ylabel('max_depth')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
for i in learning_r:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],4000, (1,1),10 ,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
plt.xlabel('max_features')
plt.ylabel('learning_rate')
plt.plot(learning_r, plotje)
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
for i in n_est:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],4000, (1,1),10, 0.8,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
XgBoost(df_annotations['text'],df_annotations['label'],4000, (1,1),10, 0.8,150)

## Stemming

In [ ]:
plotje=[]
max_feat_list=[1000, 2000, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_joined'].astype('U'),df_annotations['label'], i, (1,1))
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_joined'].astype('U'),df_annotations['label'], 3000, i)
    

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in max_depth:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_joined'].astype('U'),df_annotations['label'], 3000, (1,1),i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
for i in learning_r:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_joined'].astype('U'),df_annotations['label'], 3000, (1,1),10,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('learning_rate')
plt.ylabel('Accuracy')
plt.plot(learning_r, plotje)
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
for i in n_est:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_joined'].astype('U'),df_annotations['label'], 3000, (1,1),10,0.5, i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
XgBoost(df_annotations['processed_annotations_joined'].astype('U'),df_annotations['label'], 3000, (1,1),10,0.5, 150)

## Lemmatization

In [ ]:
plotje=[]
max_feat_list=[1000, 2000, 3000, 4000,5000 , 6000, 8000, 10000, 12000]
for i in max_feat_list:
    print('the number of max_features is', i)
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], i, (1,1))
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_feat_list, plotje)
plt.show()

In [ ]:
for i in [(1,1),(1,2)]:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 5000, i)
    

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
for i in max_depth:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 5000, (1,1),i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
for i in learning_r:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 5000, (1,1),10,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('learning_rate')
plt.ylabel('Accuracy')
plt.plot(learning_r, plotje)
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
for i in n_est:
    acc_score=XgBoost_val_tuning(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 5000, (1,1),10,0.5,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
XgBoost(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 5000, (1,1),10,0.5,150)

# Word2Vec

In [ ]:
def sent(data_column):
    sentences=[]
    for i in data_column:
        if type(i)!=float:
            words_in_sentence=i.split()
            sentences.append(words_in_sentence)
    return sentences

In [ ]:
def XgBoost(X, y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    Word2Vec_text_av=[]
    for i in X_test:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X_test=pd.DataFrame(Word2Vec_text_av)
    
    # Build XGBoost
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = xg_boost.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', accuracy_score(y_test, y_predict))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    #return y_predict, y_test, log_reg

In [ ]:
def XgBoost_val_tuning(X,y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    Word2Vec_text_av=[]
    for i in X_val:
        l=[]
        for word in i.split():
            if word in model.wv.key_to_index:
                l.append(list(model.wv.get_vector(word)))
            else:
                l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X_val=pd.DataFrame(Word2Vec_text_av)

    # Build a logistic regression
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost= model.fit(X_train, y_train)

    # Predict the labels 
    y_predict = xg_boost.predict(X_val)
    acc_score= accuracy_score(y_val, y_predict)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

## Textified

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000, i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
wind= [12, 15, 20]
for i in wind:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000, i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20]
for i in min_c:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000,10, i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000,10,4,16 ,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000,10,4,16, 1,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
print(plotje)

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
for i in max_depth:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000,10,4,16, 1,0.15,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
for i in learning_r:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000,10,4,16, 1,0.15,8,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.plot(learning_r, plotje)
plt.show()


In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
for i in n_est:
    acc_score=XgBoost_val_tuning(df_annotations['text'],df_annotations['label'],1000,10,4,16, 1,0.15,8,0.1,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
XgBoost(df_annotations['text'],df_annotations['label'],1000,10,4,16, 1,0.15,8,0.1,150)


## lemmatized

In [ ]:
def XgBoost_val_tuning_lem(X,y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    
    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    Word2Vec_text_av=[]
    for i in X_val:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        #a = np.array(l)
        #print(a)
        #res = np.average(a, axis=0)
        #lijst=list(res)
        Word2Vec_text_av.append(avg)
    X_val=pd.DataFrame(Word2Vec_text_av)

    # Build an XGBoost
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost= model.fit(X_train, y_train)

    # Predict the labels 
    y_predict = xg_boost.predict(X_val)
    acc_score= accuracy_score(y_val, y_predict)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

In [ ]:
def XgBoost_lem(X, y_inputkolom, vector_s, wind=5, min_c=1, work=4, sg=1, alpha= 0.025, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    model= Word2Vec(sentences=sent(X_train), vector_size=vector_s, window=wind, min_count=min_c, workers=work, sg=sg, alpha=alpha)
    Word2Vec_text_av=[]
    for i in X_train:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X_train=pd.DataFrame(Word2Vec_text_av)
    
    Word2Vec_text_av=[]
    for i in X_test:
        l=[]
        if type(i)== float:
            l.append([0]*vector_s)
        else:
            for word in i.split():
                if word in model.wv.key_to_index:
                    l.append(list(model.wv.get_vector(word)))
                else:
                    l.append([0]*vector_s)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X_test=pd.DataFrame(Word2Vec_text_av)
    
    # Build XGBoost
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = xg_boost.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', accuracy_score(y_test, y_predict))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    #return y_predict, y_test, log_reg

In [ ]:
plotje=[]
max_vect= [50,100,200,300,500,600,700,1000]
for i in max_vect:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.plot(max_vect, plotje)
plt.show()

In [ ]:
plotje=[]
wind= [1,2,3,4,5,6,7,8,9,10, 12, 15, 20]
for i in wind:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('window')
plt.ylabel('Accuracy')
plt.plot(wind, plotje)
plt.show()

In [ ]:
plotje=[]
min_c= [0,1,2,3,4,5,6,7,8,9,10,20]
for i in min_c:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15, i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('min_count')
plt.ylabel('Accuracy')
plt.plot(min_c, plotje)
plt.show()

In [ ]:
plotje=[]
for i in [0,1]:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15,6,16, i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
plotje=[]
alphas= [0.01,0.025,0.05,0.1,0.15,0.2,0.3,0.4,0.5,0.8,1]
for i in alphas:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15,6,16,1, i)
    plotje.append(acc_score)
    print(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.plot(alphas, plotje)
plt.show()

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
for i in max_depth:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15,6,16,1,0.1, i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
for i in learning_r:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15,6,16,1, 0.1,9,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('learning_rate')
plt.ylabel('Accuracy')
plt.plot(learning_r, plotje)
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
for i in n_est:
    acc_score=XgBoost_val_tuning_lem(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15,6,16,1, 0.1,9,0.2,i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
acc_score=XgBoost(df_annotations['processed_annotations_lemmatized_joined'].astype('U'),df_annotations['label'], 300,15,6,16,1, 0.1,9,0.2,150)
    

# Word2Vec pretrained

In [ ]:
def XgBoost(X, y_inputkolom, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Build a logistic regression
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost=model.fit(X_train, y_train)
    
    # Predict the labels 
    y_predict = xg_boost.predict(X_test)
    new=[]
    for i in y_test:
        new.append(int(i))
        
    y_pred=[]
    for i in y_predict:
        y_pred.append(int(i))
    print(' The Macro F1 measure is', metrics.f1_score(new, y_pred, average='macro'))
    print(' The Macro precision is',metrics.precision_score(new, y_pred, average='macro'))
    print(' The Macro recall is',metrics.recall_score(new, y_pred, average='macro'))
        
    print('The Accuracy of the model is', accuracy_score(y_test, y_predict))
    print('The confusion matrix looks as follows', cf_mat(y_test, y_predict))
    #return y_predict, y_test, log_reg

In [ ]:
def XgBoost_val_tuning(X,y_inputkolom, max_depth=3, learning_rate=0.1, n_est=100):
    y= y_set(y_inputkolom)
    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
    

    # Build a logistic regression
    model = XGBClassifier(use_label_encoder=False, max_depth= max_depth, learning_rate= learning_rate, n_estimators= n_est)
    xg_boost= model.fit(X_train, y_train)

    # Predict the labels 
    y_predict = xg_boost.predict(X_val)
    acc_score= accuracy_score(y_val, y_predict)
    print('The Accuracy of the model is', acc_score)
    #print('The confusion matrix looks as follows', cf_mat(y_val, y_predict))
    return acc_score

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [ ]:
def W2V_pretrained_emb(data):
    Word2Vec_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                if word in model.key_to_index:
                    l.append(list(model.get_vector(word)))
                else:
                    l.append([0]*300)
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        Word2Vec_text_av.append(avg)
    X=pd.DataFrame(Word2Vec_text_av)
    return X

## Text

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
X=W2V_pretrained_emb(df_annotations['text'])
for i in max_depth:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], max_depth=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
X=W2V_pretrained_emb(df_annotations['text'])
for i in learning_r:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], 9,learning_rate=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
objects=learning_r
y_pos= np.arange(len(objects))
plt.bar(y_pos, plotje, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.xlabel('learning_rate')
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
X=W2V_pretrained_emb(df_annotations['text'])
for i in n_est:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], 9,learning_rate=0.1, n_est= i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
X=W2V_pretrained_emb(df_annotations['text'])
acc_score=XgBoost(X, df_annotations['label'], 9, learning_rate=0.1, n_est= 100)

## Lemmatized

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in max_depth:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], max_depth=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in learning_r:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'],10, learning_rate=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
objects=learning_r
y_pos= np.arange(len(objects))
plt.bar(y_pos, plotje, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.xlabel('learning_rate')
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150,200,250,300,350,400]
X= W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in n_est:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], 10,0.1, n_est= i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
plotje=[]
n_est= [350,400,450,500,550,600]
X= W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in n_est:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'],10,0.1, n_est= i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
X=W2V_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
acc_score=XgBoost(X, df_annotations['label'],10,0.1, 120)


# fasttext pretrained

In [ ]:
import fasttext
import fasttext.util
model = fasttext.load_model('cc.nl.300.bin')

In [ ]:
def fasttext_pretrained_emb(data):
    fasttext_text_av=[]
    for i in range(len(data)):
        l=[]
        
        if type(data[i])== float:
            l.append([0]*300)
        else:
            for word in data[i].split():
                l.append(list(model.get_word_vector(word)))
        avg = [float(sum(col))/len(col) for col in zip(*l)]
        fasttext_text_av.append(avg)
    X=pd.DataFrame(fasttext_text_av)
    return X

## Text

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
X=fasttext_pretrained_emb(df_annotations['text'])
for i in max_depth:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'],max_depth=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
X=fasttext_pretrained_emb(df_annotations['text'])
for i in learning_r:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'],9, learning_rate=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
objects=learning_r
y_pos= np.arange(len(objects))
plt.bar(y_pos, plotje, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.xlabel('learning_rate')
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
X=fasttext_pretrained_emb(df_annotations['text'])
for i in n_est:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], 9, learning_rate=0.1, n_est= i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
X=fasttext_pretrained_emb(df_annotations['text'])
acc_score=XgBoost(X, df_annotations['label'],9, learning_rate=0.1, n_est= 120)

## Lemmatized

In [ ]:
plotje=[]
max_depth= [1, 2, 3, 4, 5,6,7,8,9,10]
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in max_depth:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'],i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.plot(max_depth, plotje)
plt.show()

In [ ]:
plotje=[]
learning_r= [0.01, 0.05, 0.1, 0.2, 0.5, 0.8, 1,1.2, 1.5]
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in learning_r:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'], 8,learning_rate=i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
objects=learning_r
y_pos= np.arange(len(objects))
plt.bar(y_pos, plotje, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.xlabel('learning_rate')
plt.show()

In [ ]:
plotje=[]
n_est= [10, 50, 80, 100, 120, 150]
X= fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
for i in n_est:
    acc_score=XgBoost_val_tuning(X, df_annotations['label'],8, 0.1, n_est= i)
    plotje.append(acc_score)
print(plotje)

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel('n_est')
plt.ylabel('Accuracy')
plt.plot(n_est, plotje)
plt.show()

In [ ]:
X=fasttext_pretrained_emb(df_annotations['processed_annotations_lemmatized_joined'])
acc_score=XgBoost(X, df_annotations['label'],8,0.1,150)
